# Chapter 39: ctypes Basics

This notebook covers the fundamentals of Python's `ctypes` module, which provides C-compatible
data types and allows calling functions in shared libraries. You will learn how to create C-style
integers, floats, and characters, work with string buffers, use pointers, and inspect type sizes.

## Key Concepts
- **`c_int`, `c_float`, `c_double`**: C-compatible numeric types
- **`c_char`, `c_char_p`**: C-compatible character and string types
- **`create_string_buffer`**: Mutable character buffers for C interop
- **`pointer` and `POINTER`**: Pointer creation and dereferencing
- **`sizeof`**: Query the size in bytes of C types and instances

## Section 1: C-Compatible Integer Types

The `ctypes` module provides integer types that match C's type system. Each type wraps a
Python value in a C-compatible representation. The `.value` attribute gets or sets the
underlying Python value.

In [ ]:
import ctypes

# Create a C int (typically 4 bytes / 32 bits)
i: ctypes.c_int = ctypes.c_int(42)
print(f"c_int value: {i.value}")
print(f"Type: {type(i)}")

# Modify the value through .value
i.value = 100
print(f"Updated value: {i.value}")

# Other integer types
short: ctypes.c_short = ctypes.c_short(32000)
long: ctypes.c_long = ctypes.c_long(1_000_000)
uint: ctypes.c_uint = ctypes.c_uint(42)

print(f"\nc_short: {short.value}")
print(f"c_long:  {long.value}")
print(f"c_uint:  {uint.value}")

In [ ]:
import ctypes

# Integer overflow wraps around in C types (just like C)
max_short: ctypes.c_short = ctypes.c_short(32767)
print(f"Max c_short: {max_short.value}")

# Overflow wraps to negative (signed 16-bit)
overflowed: ctypes.c_short = ctypes.c_short(32768)
print(f"32768 as c_short: {overflowed.value}")

# Unsigned types do not wrap to negative
unsigned: ctypes.c_ushort = ctypes.c_ushort(65535)
print(f"\nMax c_ushort: {unsigned.value}")

wrapped_unsigned: ctypes.c_ushort = ctypes.c_ushort(65536)
print(f"65536 as c_ushort: {wrapped_unsigned.value}")

## Section 2: C-Compatible Float Types

The `c_float` type is a single-precision (32-bit) floating-point number, while `c_double`
is double-precision (64-bit). Note that `c_float` has limited precision compared to
Python's native `float` (which is equivalent to `c_double`).

In [ ]:
import ctypes

# c_float is single precision (32-bit)
f: ctypes.c_float = ctypes.c_float(3.14)
print(f"c_float value: {f.value}")
print(f"c_float precision loss: {f.value} != 3.14 -> {f.value != 3.14}")
print(f"Close enough (abs diff < 0.01): {abs(f.value - 3.14) < 0.01}")

# c_double is double precision (64-bit) -- matches Python's float
d: ctypes.c_double = ctypes.c_double(3.14)
print(f"\nc_double value: {d.value}")
print(f"c_double exact match: {d.value == 3.14}")

# c_longdouble for extended precision (platform-dependent)
ld: ctypes.c_longdouble = ctypes.c_longdouble(3.14)
print(f"\nc_longdouble value: {ld.value}")

## Section 3: Character and Byte Types

The `c_char` type represents a single C `char` (1 byte). The `c_char_p` type is a pointer
to a null-terminated byte string, similar to `const char*` in C. The `c_wchar` type
handles wide (Unicode) characters.

In [ ]:
import ctypes

# c_char holds a single byte
ch: ctypes.c_char = ctypes.c_char(b"A")
print(f"c_char value: {ch.value}")
print(f"As integer: {ord(ch.value)}")

# c_byte holds a small signed integer (-128 to 127)
b: ctypes.c_byte = ctypes.c_byte(65)
print(f"\nc_byte value: {b.value}")
print(f"As char: {chr(b.value)}")

# c_char_p is a pointer to a byte string
s: ctypes.c_char_p = ctypes.c_char_p(b"hello")
print(f"\nc_char_p value: {s.value}")
print(f"Type: {type(s.value)}")

## Section 4: String Buffers

The `create_string_buffer` function creates a mutable character buffer. This is
essential when you need to pass a writable buffer to a C function. The buffer
has a fixed size and stores bytes.

In [ ]:
import ctypes

# Create a buffer with a fixed size (10 bytes, initialized to zero)
buf: ctypes.Array[ctypes.c_char] = ctypes.create_string_buffer(10)
print(f"Buffer length: {len(buf)}")
print(f"Initial value: {buf.value!r}")
print(f"Raw bytes: {buf.raw!r}")

# Set the buffer value
buf.value = b"hello"
print(f"\nAfter setting 'hello':")
print(f"Value: {buf.value}")
print(f"Raw: {buf.raw!r}")
print(f"Length still: {len(buf)}")

In [ ]:
import ctypes

# Create a buffer initialized with a byte string
buf: ctypes.Array[ctypes.c_char] = ctypes.create_string_buffer(b"world")
print(f"Value: {buf.value}")
print(f"Length: {len(buf)}")  # 6: 5 chars + null terminator

# Create with initial value AND a minimum size
buf2: ctypes.Array[ctypes.c_char] = ctypes.create_string_buffer(b"hi", 20)
print(f"\nValue: {buf2.value}")
print(f"Length: {len(buf2)}")

# Buffers are mutable -- you can modify individual bytes
buf2[0] = b"H"
print(f"After modifying index 0: {buf2.value}")

## Section 5: Pointers

Pointers are fundamental to C interoperability. The `pointer()` function creates a
pointer to a ctypes instance, and the `POINTER()` function creates a pointer *type*.
Use `.contents` to dereference a pointer.

In [ ]:
import ctypes

# Create a c_int and a pointer to it
i: ctypes.c_int = ctypes.c_int(42)
p = ctypes.pointer(i)

print(f"Original value: {i.value}")
print(f"Pointer contents: {p.contents.value}")
print(f"Pointer type: {type(p)}")

# Modify through the pointer
p.contents.value = 100
print(f"\nAfter modifying through pointer:")
print(f"Original i.value: {i.value}")
print(f"Pointer contents: {p.contents.value}")
print(f"Both point to same data: {i.value == p.contents.value}")

In [ ]:
import ctypes

# POINTER() creates a pointer TYPE (not an instance)
IntPointer = ctypes.POINTER(ctypes.c_int)
print(f"Pointer type: {IntPointer}")

# Create an instance of the pointer type
val: ctypes.c_int = ctypes.c_int(99)
ptr: ctypes.POINTER(ctypes.c_int) = IntPointer(val)
print(f"Value via pointer: {ptr.contents.value}")

# Null pointer (no argument)
null_ptr = IntPointer()
print(f"\nNull pointer created (accessing contents would raise ValueError)")

# byref() is an optimization for passing pointers to C functions
x: ctypes.c_int = ctypes.c_int(77)
ref = ctypes.byref(x)
print(f"byref type: {type(ref)}")

## Section 6: sizeof -- Inspecting Type Sizes

The `ctypes.sizeof()` function returns the size in bytes of a ctypes type or instance.
This mirrors C's `sizeof` operator and is essential for understanding memory layout.

In [ ]:
import ctypes

# sizeof on types (class-level)
print("Type sizes (in bytes):")
print(f"  c_char:      {ctypes.sizeof(ctypes.c_char)}")
print(f"  c_short:     {ctypes.sizeof(ctypes.c_short)}")
print(f"  c_int:       {ctypes.sizeof(ctypes.c_int)}")
print(f"  c_long:      {ctypes.sizeof(ctypes.c_long)}")
print(f"  c_longlong:  {ctypes.sizeof(ctypes.c_longlong)}")
print(f"  c_float:     {ctypes.sizeof(ctypes.c_float)}")
print(f"  c_double:    {ctypes.sizeof(ctypes.c_double)}")

# Verify expected sizes
print(f"\nc_int is 4 bytes: {ctypes.sizeof(ctypes.c_int) == 4}")
print(f"c_double is 8 bytes: {ctypes.sizeof(ctypes.c_double) == 8}")
print(f"c_char is 1 byte: {ctypes.sizeof(ctypes.c_char) == 1}")

In [ ]:
import ctypes

# sizeof also works on instances
i: ctypes.c_int = ctypes.c_int(42)
d: ctypes.c_double = ctypes.c_double(3.14)

print(f"sizeof(c_int instance): {ctypes.sizeof(i)}")
print(f"sizeof(c_double instance): {ctypes.sizeof(d)}")

# sizeof on a string buffer
buf: ctypes.Array[ctypes.c_char] = ctypes.create_string_buffer(10)
print(f"sizeof(10-byte buffer): {ctypes.sizeof(buf)}")

# sizeof on an array type
IntArray5 = ctypes.c_int * 5
print(f"\nsizeof(c_int * 5): {ctypes.sizeof(IntArray5)}")
print(f"Expected (5 * 4): {5 * ctypes.sizeof(ctypes.c_int)}")

## Section 7: Comparing ctypes with Python Native Types

Understanding when to use ctypes versus native Python types is important. ctypes
types are needed specifically for C library interoperability, not for general Python programming.

In [ ]:
import ctypes

# ctypes values are not the same as Python values
c_val: ctypes.c_int = ctypes.c_int(42)
py_val: int = 42

print(f"ctypes c_int: {c_val}, type: {type(c_val)}")
print(f"Python int:   {py_val}, type: {type(py_val)}")
print(f"Values equal: {c_val.value == py_val}")

# Convert between ctypes and Python
from_c: int = c_val.value  # ctypes -> Python
to_c: ctypes.c_int = ctypes.c_int(py_val)  # Python -> ctypes

print(f"\nConverted from ctypes: {from_c} (type: {type(from_c)})")
print(f"Converted to ctypes: {to_c.value} (type: {type(to_c)})")

# Summary of type mapping
mappings: list[tuple[str, str]] = [
    ("c_int", "int"),
    ("c_float", "float (single precision)"),
    ("c_double", "float (double precision)"),
    ("c_char", "bytes (1 byte)"),
    ("c_char_p", "bytes or None"),
    ("c_bool", "bool"),
]
print(f"\n{'ctypes Type':<15} {'Python Type'}")
print("-" * 40)
for c_type, py_type in mappings:
    print(f"{c_type:<15} {py_type}")

## Summary

### Fundamental Types
- **`c_int(value)`**: 4-byte signed integer, access via `.value`
- **`c_float(value)`**: 4-byte single-precision float (limited precision)
- **`c_double(value)`**: 8-byte double-precision float (matches Python `float`)
- **`c_char(byte)`**: Single byte character
- **`c_char_p(bytes)`**: Pointer to a null-terminated byte string

### String Buffers
- **`create_string_buffer(size)`**: Mutable buffer of `size` zero bytes
- **`create_string_buffer(b"init")`**: Buffer initialized from bytes (size = len + 1)
- **`create_string_buffer(b"init", size)`**: Initialized buffer with minimum size
- Access via `.value` (null-terminated) or `.raw` (all bytes)

### Pointers
- **`pointer(obj)`**: Create a pointer instance to a ctypes object
- **`POINTER(type)`**: Create a pointer type (class)
- **`.contents`**: Dereference a pointer to access the pointed-to object
- **`byref(obj)`**: Lightweight pointer for passing to C functions

### sizeof
- **`sizeof(type_or_instance)`**: Returns size in bytes
- Works on both ctypes types and instances
- `c_char` = 1, `c_int` = 4, `c_double` = 8 bytes (typical)